In [1]:
import os
import numpy as np
import pyedflib
from scipy.stats import zscore
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report


In [2]:
# 설정
sampling_rate = 256
segment_length = 30 * sampling_rate  # 30초 = 7680포인트

label_map = {
    'Sleep stage W': 0,
    'Sleep stage N1': 1,
    'Sleep stage N2': 1,
    'Sleep stage N3': 2,
    'Sleep stage R': 3
}

def load_ecg_and_labels(sn_id, root_dir):
    base = f"SN{int(sn_id):03d}"
    ecg_path = os.path.join(root_dir, f"{base}.edf")
    label_path = os.path.join(root_dir, f"{base}_sleepscoring.edf")
    
    if not os.path.exists(ecg_path) or not os.path.exists(label_path):
        print(f"❌ 파일 없음: {base}")
        return [], []

    with pyedflib.EdfReader(ecg_path) as ecg_reader:
        ecg_signal = ecg_reader.readSignal(7)

    with pyedflib.EdfReader(label_path) as label_reader:
        onsets, durations, labels = label_reader.readAnnotations()

    segments, segment_labels = [], []

    for onset, duration, label in zip(onsets, durations, labels):
        label_str = label.decode() if isinstance(label, bytes) else label
        if label_str not in label_map:
            continue  # 'Lights on/off' 무시

        start = int(onset * sampling_rate)
        num_segments = int(duration // 30)

        for i in range(num_segments):
            seg_start = start + i * segment_length
            seg_end = seg_start + segment_length
            if seg_end > len(ecg_signal):
                break

            segment = ecg_signal[seg_start:seg_end]
            segment = zscore(segment)
            segments.append(segment.astype(np.float32))
            segment_labels.append(label_map[label_str])

    return segments, segment_labels

def create_sequences(segments, labels, seq_len=10):
    X, y = [], []
    for i in range(len(segments) - seq_len + 1):
        X.append(np.stack(segments[i:i+seq_len]))
        y.append(labels[i + seq_len - 1])
    return np.array(X), np.array(y)


In [3]:
class SleepDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [4]:
class SleepLSTM(nn.Module):
    def __init__(self, input_size=7680, hidden_size=128, num_layers=2, num_classes=4):
        super(SleepLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)


In [10]:
def train_model(model, loader, epochs=10, lr=0.0001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == y_batch).sum().item()
            total += y_batch.size(0)

        acc = correct / total if total > 0 else 0
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {total_loss:.4f} Acc: {acc:.4f}")

from sklearn.metrics import classification_report

def evaluate_one_person(model, loader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model = model.to(device)
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = outputs.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y_batch.numpy())

    # labels 인자를 추가!
    print(classification_report(
        all_labels, all_preds, 
        labels=[0,1,2,3],  # 모든 클래스 포함
        target_names=["W", "N1/2", "N3", "R"],
        zero_division=0    # division by zero 경고 안 뜨게
    ))



In [ ]:
# 데이터 로드 및 분리
root_dir = "/home/mhb0917/캡스톤디자인/sleep/recordings"
seq_len = 10

train_X, train_y = [], []
val_data, test_data = {}, {}

for i in range(1, 155):
    segments, labels = load_ecg_and_labels(i, root_dir)
    if segments:
        X_seq, y_seq = create_sequences(segments, labels, seq_len)
        if 1 <= i <= 93:
            train_X.append(X_seq)
            train_y.append(y_seq)
        elif 94 <= i <= 123:
            val_data[f"SN{i:03d}"] = (X_seq, y_seq)
        else:
            test_data[f"SN{i:03d}"] = (X_seq, y_seq)

train_X = np.concatenate(train_X)
train_y = np.concatenate(train_y)

# 학습
train_dataset = SleepDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)




❌ 파일 없음: SN014
❌ 파일 없음: SN064
❌ 파일 없음: SN135
Epoch [1/10] Loss: nan Acc: 0.1619
Epoch [2/10] Loss: nan Acc: 0.1616
Epoch [3/10] Loss: nan Acc: 0.1616
Epoch [4/10] Loss: nan Acc: 0.1616
Epoch [5/10] Loss: nan Acc: 0.1616
Epoch [6/10] Loss: nan Acc: 0.1616
Epoch [7/10] Loss: nan Acc: 0.1616
Epoch [8/10] Loss: nan Acc: 0.1616
Epoch [9/10] Loss: nan Acc: 0.1616
Epoch [10/10] Loss: nan Acc: 0.1616

📊 Validation Results:

🧪 Evaluation for SN094
              precision    recall  f1-score   support

           W       0.07      1.00      0.13        70
        N1/2       0.00      0.00      0.00       604
          N3       0.00      0.00      0.00       202
           R       0.00      0.00      0.00       165

    accuracy                           0.07      1041
   macro avg       0.02      0.25      0.03      1041
weighted avg       0.00      0.07      0.01      1041


🧪 Evaluation for SN095
              precision    recall  f1-score   support

           W       0.49      1.00      0.66

In [12]:
model = SleepLSTM()
train_model(model, train_loader, epochs=1)

# 평가 (Validation)
import traceback
print("\n📊 validation Results:")
for sn, (X, y) in val_data.items():
    loader = DataLoader(SleepDataset(X, y), batch_size=16, shuffle=False)
    print(f"\n🧪 Evaluation for {sn} (samples: {len(y)})")
    try:
        evaluate_one_person(model, loader)
    except Exception as e:
        print(f"⚠️ Error evaluating {sn}: {e}")
        traceback.print_exc()

# 평가 (Test)
print("\n📊 Test Results:")
for sn, (X, y) in val_data.items():
    loader = DataLoader(SleepDataset(X, y), batch_size=16, shuffle=False)
    print(f"\n🧪 Evaluation for {sn} (samples: {len(y)})")
    try:
        evaluate_one_person(model, loader)
    except Exception as e:
        print(f"⚠️ Error evaluating {sn}: {e}")
        traceback.print_exc()

Epoch [1/1] Loss: nan Acc: 0.1626

📊 validation Results:

🧪 Evaluation for SN094 (samples: 1041)
              precision    recall  f1-score   support

           W       0.07      1.00      0.13        70
        N1/2       0.00      0.00      0.00       604
          N3       0.00      0.00      0.00       202
           R       0.00      0.00      0.00       165

    accuracy                           0.07      1041
   macro avg       0.02      0.25      0.03      1041
weighted avg       0.00      0.07      0.01      1041


🧪 Evaluation for SN095 (samples: 853)
              precision    recall  f1-score   support

           W       0.49      1.00      0.66       419
        N1/2       0.00      0.00      0.00       273
          N3       0.00      0.00      0.00       111
           R       0.00      0.00      0.00        50

    accuracy                           0.49       853
   macro avg       0.12      0.25      0.16       853
weighted avg       0.24      0.49      0.32      